In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx

def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["data-source", "ml"]
fn.metadata.labels = {"author": "yjb", "framework": "sklearn"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-05-03 13:04:07,376 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 13:04:07,396 function spec saved to path: function.yaml


## tests

In [8]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [9]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "iris"}}

### run remotely

In [10]:
run = fn.run(NewTask(**task_params), artifact_path=mlconf.artifact_path)

[mlrun] 2020-05-03 13:04:40,826 starting run tasks load toy dataset uid=638e4fc1ed2e4231b1d0f55988c423db  -> http://mlrun-api:8080
[mlrun] 2020-05-03 13:04:40,952 Job is running in the background, pod: tasks-load-toy-dataset-s6kbw
[mlrun] 2020-05-03 13:04:44,408 log artifact iris at /home/jovyan/data/iris.parquet, size: 5041, db: Y

[mlrun] 2020-05-03 13:04:44,419 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...c423db,0,May 03 13:04:44,completed,tasks load toy dataset,host=tasks-load-toy-dataset-s6kbwkind=jobowner=jovyan,,dataset=iris,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 638e4fc1ed2e4231b1d0f55988c423db  , !mlrun logs 638e4fc1ed2e4231b1d0f55988c423db 
[mlrun] 2020-05-03 13:04:47,082 run executed, status=completed


### or locally

In [11]:
from mlrun import run_local

In [12]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset}, artifact_path=mlconf.artifact_path)

[mlrun] 2020-05-03 13:04:47,096 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 13:04:47,096 starting run mlrun-e1eecd-load_dataset uid=03a6673b04f24ce3a2ca456e4a392673  -> http://mlrun-api:8080
[mlrun] 2020-05-03 13:04:47,110 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 13:04:47,508 log artifact wine at /home/jovyan/data/wine.parquet, size: 18454, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...392673,0,May 03 13:04:47,completed,mlrun-e1eecd-load_dataset,kind=handlerowner=jovyanhost=jupyter-notebook-7c79d94f7d-c2lr9,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 03a6673b04f24ce3a2ca456e4a392673 --project default , !mlrun logs 03a6673b04f24ce3a2ca456e4a392673 --project default
[mlrun] 2020-05-03 13:04:47,546 run executed, status=completed
[mlrun] 2020-05-03 13:04:47,553 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 13:04:47,554 starting run mlrun-da29e4-load_dataset uid=2ea4dc37dae74810be7c686e556ea116  -> http://mlrun-api:8080
[mlrun] 2020-05-03 13:04:47,569 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 13:04:47,603 log artifact iris at /home/jovyan/data/iris.parquet, size: 5209, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...6ea116,0,May 03 13:04:47,completed,mlrun-da29e4-load_dataset,kind=handlerowner=jovyanhost=jupyter-notebook-7c79d94f7d-c2lr9,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 2ea4dc37dae74810be7c686e556ea116 --project default , !mlrun logs 2ea4dc37dae74810be7c686e556ea116 --project default
[mlrun] 2020-05-03 13:04:47,628 run executed, status=completed
[mlrun] 2020-05-03 13:04:47,634 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 13:04:47,634 starting run mlrun-535410-load_dataset uid=b605376648d34fa18ffc32fef752e576  -> http://mlrun-api:8080
[mlrun] 2020-05-03 13:04:47,645 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 13:04:47,754 log artifact breast_cancer at /home/jovyan/data/breast_cancer.parquet, size: 147746, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...52e576,0,May 03 13:04:47,completed,mlrun-535410-load_dataset,kind=handlerowner=jovyanhost=jupyter-notebook-7c79d94f7d-c2lr9,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run b605376648d34fa18ffc32fef752e576 --project default , !mlrun logs b605376648d34fa18ffc32fef752e576 --project default
[mlrun] 2020-05-03 13:04:47,807 run executed, status=completed
